In [112]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
import hydroeval as he

In [113]:
pred_basel = pd.read_csv("data/basel.csv",index_col=0)
pred_basel = pred_basel.set_index("datetime")

In [114]:
q_basel = pd.read_csv("data/q_basel.csv",index_col=0)
q_basel

,obs,pcr,res
datetime,,,
1981-01-01,670.000,605.06915,64.93085
1981-01-02,647.000,599.35767,47.64233
1981-01-03,727.000,874.38354,-147.38354
1981-01-04,1363.000,998.86804,364.13196
1981-01-05,1202.000,968.06270,233.93730
...,...,...,...
2000-12-27,662.035,1055.62610,-393.59110
2000-12-28,656.253,1027.90730,-371.65430
2000-12-29,652.150,860.76306,-208.61306


In [115]:
pred_basel.columns

Index(['et', 'p', 't', 'obs'], dtype='object')

### Feature Engineering

In [116]:
# making 50 lagged variables
TIME_STEPS = 50
is_lag = TIME_STEPS > 1
if is_lag:
  #add the lagged variables to the dataframe
  for i, var in enumerate(pred_basel[["et","p","t"]]):
    for step in range(0, TIME_STEPS - 1):
      pred_basel.insert(i*(TIME_STEPS) + 1,
                        f'{var}_lag_{TIME_STEPS - 1 - step}',
                        pred_basel[var].shift(TIME_STEPS - 1 - step))

#remove the first TIME_STEPS - 1 rows since they will contain NA values
pred_basel = pred_basel.iloc[TIME_STEPS - 1:,:].reset_index(drop=True)
q_basel = q_basel.iloc[TIME_STEPS - 1:,:].reset_index(drop=True)

C:\Users\Wescap\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3361: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


In [117]:
pred_basel.head()


,et,et_lag_1,et_lag_2,et_lag_3,et_lag_4,et_lag_5,et_lag_6,et_lag_7,et_lag_8,et_lag_9,...,t_lag_41,t_lag_42,t_lag_43,t_lag_44,t_lag_45,t_lag_46,t_lag_47,t_lag_48,t_lag_49,obs
0,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,0.000483,0.000623,0.000770,...,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,5.165919,2.172385,-0.250816,704.0
1,0.000547,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,0.000483,0.000623,...,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,5.165919,2.172385,693.0
2,0.000569,0.000547,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,0.000483,...,-5.925995,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,5.165919,659.0
3,0.000593,0.000569,0.000547,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,...,-5.499934,-5.925995,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,652.0
4,0.000644,0.000593,0.000569,0.000547,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,...,-4.769065,-5.499934,-5.925995,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,630.0


### train-test split

In [118]:
# predictors and output var
len_ = int(0.75 * pred_basel.shape[0])

df_train = pred_basel[:len_]
df_test = pred_basel[len_:]


X_train = df_train.drop("obs", axis =1)
y_train = df_train.obs

X_test = df_test.drop("obs", axis =1)
y_test = df_test.obs


In [119]:
X_train.head(3)

,et,et_lag_1,et_lag_2,et_lag_3,et_lag_4,et_lag_5,et_lag_6,et_lag_7,et_lag_8,et_lag_9,...,t_lag_40,t_lag_41,t_lag_42,t_lag_43,t_lag_44,t_lag_45,t_lag_46,t_lag_47,t_lag_48,t_lag_49
0,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,0.000483,0.000623,0.000770,...,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,5.165919,2.172385,-0.250816
1,0.000547,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,0.000483,0.000623,...,-5.925995,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,5.165919,2.172385
2,0.000569,0.000547,0.000532,0.000553,0.000571,0.000580,0.000543,0.000507,0.000452,0.000483,...,-5.499934,-5.925995,-5.646553,-8.999503,-10.731832,-6.366458,-2.218504,-2.055391,1.019687,5.165919


### Normalise data

In [120]:
# normalising

# scaler = MinMaxScaler()
#
# X_train_scaled = scaler.fit_transform(X_train)
#
# X_test_scaled = scaler.transform(X_test)

# no normalisation is needed  xgboost website: https://github.com/dmlc/xgboost/issues/357
# https://datascience.stackexchange.com/questions/60950/is-it-necessary-to-normalize-data-for-xgboost#:~:text=Your%20rationale%20is%20indeed%20correct,normalization%20for%20the%20inputs%20either.


### Hyperparameter grid search

In [121]:
%%time
# https://www.kaggle.com/code/jayatou/xgbregressor-with-gridsearchcv/script
# what is each parameter doing? : https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

# Various hyper-parameters to tune

xgb = XGBRegressor()
parameters = {'nthread':[1], # when use hyperthread, xgboost may become slower
              'objective':['reg:linear'], # maybe should change with this: reg:squarederror
              'learning_rate': [.01, .05, .1], # so called `eta` value # reasonable range is : 0.01 – 0.3
              'max_depth': [7, 9], # reasonable range is 3 to 10
              'min_child_weight': [4],
              'subsample': [0.8, 0.9], # it is data to be sampled for each tree # reasonable range is : 0.6 to 1
              'colsample_bytree': [0.7, 0.9], # the fraction of columns to be randomly sampled for each tree # reasonable range is : 0.5 to 1
              'n_estimators': [800, 1000]} # number of trees if dataset is small 1000 if it is large 100

xgb_grid = GridSearchCV(xgb,
                        parameters,
                        cv = 5, #maybe 2 #maybe leave out colsample above
                        n_jobs = 5,
                        verbose=True)


xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[22:52:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
0.6327245517460435
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 800, 'nthread': 1, 'objective': 'reg:linear', 'subsample': 0.8}
Wall time: 3h 33min 39s


In [123]:
# read the results in a df or csv and save it somewhere so that you can load it
# sav ethe best model
# run xgboost without lagged variable
# maybe if it takes a long time run it with 10 lagged variable

dict_ = xgb_grid.best_params_
df_best_params = pd.DataFrame([dict_])
df_best_params

,colsample_bytree,learning_rate,max_depth,min_child_weight,n_estimators,nthread,objective,subsample
0,0.9,0.05,7,4,800,1,reg:linear,0.8


In [124]:
df_best_params.to_csv('xgboost_best_params.csv', index=False)

### Model

In [125]:
# define the model
model =  xgb_grid.best_estimator_

# fit model
model.fit(X_train, y_train)
# make a prediction
yhat = model.predict(X_test)

[23:18:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [126]:
### save model
model.save_model('xgboost_model.model')


###  Random Model

In [74]:
# # create a xgboost regression model
# model = xgb.XGBRegressor(n_estimators=1000, max_depth=20, eta=0.1, subsample=0.7, colsample_bytree=0.8)


In [75]:
# # define model evaluation method
# cv = RepeatedKFold(n_splits=10, n_repeats=7, random_state=7)
# # evaluate model
# scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [76]:
# # define model
# model = xgb.XGBRegressor(n_estimators=1000, max_depth=20, eta=0.1, subsample=0.7, colsample_bytree=0.8)
# # fit model
# model.fit(X_train, y_train)
# # make a prediction
# yhat = model.predict(X_test)

### Evaluation

In [127]:
streamflow_pcr = q_basel.pcr[len_:]

#evaluate pcr model using nse and kge
nse_pcr = he.evaluator(he.nse, streamflow_pcr,y_test)
kge_pcr, r, alpha, beta = he.evaluator(he.kge, streamflow_pcr, y_test)
print("The nse and kge of the PCR model are {:.2} and {:.2}.".format(nse_pcr[0], kge_pcr[0]))

#evaluate the prediction using nse and kge
nse = he.evaluator(he.nse, yhat, y_test)

kge, r, alpha, beta = he.evaluator(he.kge, yhat, y_test)
print("The nse and kge of the xgboost model are {:.2} and {:.2}.".format(nse[0], kge[0]))

The nse and kge of the PCR model are 0.22 and 0.64.
The nse and kge of the xgboost model are 0.6 and 0.61.
